## estimate gas-phase ammonia concentration, as well as nhx in the droplet. get an initial estimate of the droplet ph. estimate vial nh3.

In [20]:
from scipy.optimize import fsolve
import numpy as np

11-04-2019

#### assumptions: ions present are hydroxyl, hydrogen, sulfate, ammonium. neglect acetic acid. assume ph range high enough that there is no sulfuric acid present.

Charge balance: $ 2*[SO_4^{2-}] + [HSO_4^-] + [OH^-] = [H^+] + [NH_4^+] $

Acid-base equilibria: $ K_{a,N} = \frac{[NH_3][H^+]}{[NH_4^+]} $ and $ K_{a,S2} = \frac{[SO_4^{2-}][H^+]}{[HSO_4^-]} $ 

Sulfur balance: $ S_T = [SO_4^{2-}] + [HSO_4^-] $

NHX balance: $ NH_T = [NH_4^{+}] + [NH_3] $

Water equilibrium: $ K_w = [OH^-][H^+] $ so $ [OH^-] = \frac{K_w}{[H^+]} $

---

get sulfur species in terms of $ [SO_4^{2-}] $ so $ [HSO_4^-] = \frac{[H^+][SO_4^{2-}]}{K_{a,s2}} $

Now $ S_T $ can be expressed in terms of $ [SO_4^{2-}] $ : $ S_T = [SO_4^{2-}] + \frac{[H^+][SO_4^{2-}]}{K_{a,s2}} $ and as such: $ [SO_4^{2-}] = S_T*(\frac{[H^+]}{K_{a,S2}} + 1)^{-1} $

Similarly, can do this with NHX. $ [NH_3] = \frac{[NH_4^+] K_a}{[H^+]} $ and so $ [NH_4^+] = [NH_T](1 + \frac{K_a}{[H^+]})^{-1} $

Finally, all the above can be plugged into the charge balance in terms of constants and H+.

$ 2*[SO_4^{2-}] + (S_T - [SO_4^{2-}]) + [OH^-] = [H^+] + [NH_4^+] $
pH can be solved for with python.

In [21]:
def calc_as_ph(ammsulf):
    s_tot = ammsulf
    nh_tot = 2*ammsulf
    k_w = 1e-14
    k_n = 5.8e-10
    k_s = 1.2e-2
    
    def find_h(h):
        return h + (nh_tot * (1 + k_n/h)**-1) - (k_w / h) - s_tot * (1 + ((h / k_s) + 1)**-1)
    
    h = fsolve(find_h, 1e-7)
    ph = -np.log10(h[0])

    return ph

In [22]:
def calc_ah_ph(ammhyd):
    nh_tot = ammhyd
    k_w = 1e-14
    k_n = 5.8e-10
    
    def find_oh(oh):
        return (k_w/oh) + (nh_tot * (1 + k_n/(k_w/oh))**-1) - oh
    
    oh = fsolve(find_oh, 1e-9)
    ph = 14+np.log10(oh[0])

    return ph

In [23]:
ammsulf = 0.32
ph = calc_as_ph(ammsulf)
ph

5.4361390810330015

In [24]:
ammhyd = 0.32
ph = calc_ah_ph(ammhyd)
ph

11.369267080723459

In [25]:
def calc_nh3(ph, nh_tot):
    h = 10**-ph
    k_n = 5.8e-10
    
    return nh_tot * (h / k_n + 1)**-1

In [34]:
nh3 = calc_nh3(4, 0.58)
nh3

3.3639804889131636e-06

In [27]:
def calc_nhx(ph, nh3):
    h = 10**-ph
    k_n = 5.8e-10
    
    return nh3 * (1 + (h / k_n))

In [28]:
nhx = calc_nhx(8.5, 0.54)
nhx

3.484189545674009

In [29]:
0.52/0.0081

64.19753086419753

In [30]:
# mol nhx / kg h2o * g as / mol nhx * kg h2o = g as

0.0081 * (132/2) * .65e-3 * 2

0.0006949799999999999

In [31]:
# calculate molarities from rh and composition

x_h2o = 0.75
x_other = 1 - x_h2o
bd_peg6_molar_ratio = (0.2 / 282) / (.075 / 120)
x_bd = x_other * (1 + bd_peg6_molar_ratio) ** -1
x_peg = x_other - x_bd

In [32]:
(x_bd / x_h2o) * 1000 / 18

8.674787744555186